<a href="https://colab.research.google.com/github/rodr1ggoql17/Sistema-de-base-de-datos/blob/main/clavesSQL_basicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3 as sq

In [ ]:
conexion = sq.connect('usuarios.db')
cursor = conexion.cursor()

In [ ]:
# creamos un campo dni como clave primaria
cursor.execute('''CREATE TABLE IF NOT EXISTS usuarios (
  dni VARCHAR(9) PRIMARY KEY,
  nombre VARCHAR(100),
  edad INTEGER,
  email VARCHAR(100))''')

usuarios = [('11111111A','Hector',27,'hector@ejemplo.com'),
            ('22222222B','Mario',51,'mario@ejemplo.com'),
            ('33333333C','Mercedes',38,'mercedes@ejemplo.com'),
            ('44444444D','Rodrigo',20,'rreyesm@nasa.eu')]
cursor.executemany("INSERT INTO usuarios VALUES (?,?,?,?)",usuarios)
conexion.commit()

In [ ]:
# comprobamos que al ingresar un dato duplicado nos arroja un error (esto es dado que la primary key se repite)
cursor = conexion.cursor()
# Añadimos un usuario con el mismo DNI
cursor.execute("INSERT INTO usuarios VALUES " \
    "('11111111A', 'Fernando', 31, 'fernando@ejemplo.com')")
conexion.commit()

IntegrityError: ignored

#Claves autoincrementales
No siempre contaremos con claves primarias en nuestras tablas (como el DNI), sin embargo siempre necesitaremos uno para identificar cada registro y poder consultarlo, modificarlo o borrarlo.

Para estas situaciones lo más útil es utilizar campos autoincrementales, campos especiales que asignan automáticamente un número (de uno en uno) al crear un nuevo registro. Es muy útil para identificar de forma única cada registro ya que nunca se repiten.

En SQLite, si indicamos que un campo numérico entero es una clave primaria, automáticamente se tomará como un campo auto incremental. Podemos hacerlo fácilmente así:


In [ ]:
# creamos una base de datos de productos
conexion = sq.connect('productos.db')
cursor = conexion.cursor()

# not null -> indica que los campos no pueden ser vacios
cursor.execute('''
    CREATE TABLE IF NOT EXISTS productos(
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      nombre VARCHAR(100) NOT NULL,
      marca VARCHAR(50) NOT NULL,
      precio FLOAT NOT NULL)''')

#¡Problema al insertar registros con campos autoincrementales!

Al utilizar un nuevo campo autoincremental, la sintaxis sencilla para insertar registros ya no funciona, pues en primer lugar se espera un identificador único, por lo que recibimos un error indicándonos se esperan 4 columnas en lugar de 3:

In [ ]:
cursor = conexion.cursor()
productos = [('Teclado', 'Logitech', 19.95),
            ('Pantalla 19"' 'LG', 89.95),]
cursor.executemany("INSERT INTO productos VALUES (?,?,?)", productos)
conexion.commit()

OperationalError: ignored

# Para resolver este problema

In [ ]:
cursor = conexion.cursor()
productos = [('Teclado', 'Logitech', 19.95),
            ('Pantalla 19"','LG', 89.95),
            ('Altavoces 2.1','LG', 24.95),]
cursor.executemany("INSERT INTO productos VALUES (null,?,?,?)",productos)
conexion.commit()

In [ ]:
# ahora podemos consultar productos por su identificador unico
cursor = conexion.cursor()
cursor.execute("SELECT * FROM productos")
productos = cursor.fetchall()
conexion.commit()

In [ ]:
# ver
productos

[(1, 'Teclado', 'Logitech', 19.95),
 (2, 'Pantalla 19"', 'LG', 89.95),
 (3, 'Altavoces 2.1', 'LG', 24.95)]

# Claves únicas
El problema con las claves primarias es que sólo podemos tener un campo con esta propiedad, y si da la casualidad que utilizamos un campo autoincremental, ya no podemos asignarla a otro campo.

Para estos casos existen las claves únicas, que nos permiten añadir otros campos únicos no repetibles.

Podemos adaptar el ejemplo de los usuarios con un campo autoincremental que haga de clave primaria, y a su vez marcar el DNI como un campo único:

In [ ]:
cursor = conexion.cursor()

# dni es primary key
cursor.execute('''CREATE TABLE IF NOT EXISTS usuarios (
                    id INTEGER PRIMARY KEY,
                    dni VARCHAR(9) UNIQUE,
                    nombre VARCHAR(100), 
                    edad INTEGER(3),
                    email VARCHAR(100))''')
usuarios = [('11111111A', 'Hector', 27, 'hector@ejemplo.com'),
            ('22222222B', 'Mario', 51, 'mario@ejemplo.com'),
            ('33333333C', 'Mercedes', 38, 'mercedes@ejemplo.com'),
            ('44444444D', 'Juan', 19, 'juan@ejemplo.com')]

cursor.executemany("INSERT INTO usuarios VALUES (null, ?,?,?,?)", usuarios)
conexion.commit()

In [ ]:
# si ingresamos un usuario con la misma clave nos arroja error
cursor = conexion.cursor()
cursor.execute("INSERT INTO usuarios VALUES " \
    "(null, '11111111A', 'Fernando', 31, 'fernando@ejemplo.com')")

conexion.commit()

IntegrityError: ignored

In [ ]:
usuarios

[('11111111A', 'Hector', 27, 'hector@ejemplo.com'),
 ('22222222B', 'Mario', 51, 'mario@ejemplo.com'),
 ('33333333C', 'Mercedes', 38, 'mercedes@ejemplo.com'),
 ('44444444D', 'Juan', 19, 'juan@ejemplo.com')]

In [ ]:
conexion = sq.connect('productos.db')
cursor = conexion.cursor()
# con la ventaja de contar con un identificador automático para cada registro:
cursor = conexion.cursor()
# Recuperamos los registros de la tabla de usuarios
cursor.execute("SELECT * FROM usuarios")

# Recorremos todos los registros con fetchall
# y los volcamos en una lista de usuarios
usuarios = cursor.fetchall()
usuarios

[(1, '11111111A', 'Hector', 27, 'hector@ejemplo.com'),
 (2, '22222222B', 'Mario', 51, 'mario@ejemplo.com'),
 (3, '33333333C', 'Mercedes', 38, 'mercedes@ejemplo.com'),
 (4, '44444444D', 'Juan', 19, 'juan@ejemplo.com')]